In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer

# Khởi tạo Spark Session
print("Initializing Spark Session...")
spark = SparkSession.builder \
    .appName("Real Estate Analysis") \
    .config("spark.driver.memory", "8g") \
    .config("spark.jars.packages", "ml.dmlc:xgboost4j-spark_2.12:1.5.2") \
    .master("local[*]") \
    .getOrCreate()

# ====================== PHẦN TIỀN XỬ LÝ DỮ LIỆU ======================
# Đọc dữ liệu
print("Đang đọc dữ liệu...")
df = spark.read.csv("realtor-data.zip.csv", header=True, inferSchema=True)

# Kiểm tra thông tin dữ liệu gốc
df.printSchema()
# Loại bỏ các cột không cần thiết
df = df.drop("city", "zip_code", "prev_sold_date")
# Kiểm tra số lượng giá trị null
null_counts = df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in df.columns])
print("\nSố lượng giá trị null:")
null_counts.show()



# Loại bỏ các hàng có giá trị null trong cột quan trọng
df = df.filter(~col("price").isNull())
df = df.filter(~col("bed").isNull())
df = df.filter(~col("bath").isNull())

# Xử lý cột status
print("\nGiá trị trong cột status:")
df.groupBy("status").count().show()
df = df.drop("status")

# Xử lý cột state
print("\nGiá trị trong cột state:")
df.groupBy("state").count().show()

# Chỉ giữ lại các state có ít nhất 50 mẫu
state_counts = df.groupBy("state").count()
valid_states = state_counts.filter(col("count") >= 50).select("state")
df = df.join(valid_states, "state", "inner")

# Mã hóa state thành số
indexer = StringIndexer(inputCol="state", outputCol="state_numeric", handleInvalid="skip")
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)
df = df.drop("state")

# Tạo mapping từ số đến tên state
state_labels = indexer_model.labels
numeric_to_state = {i: state for i, state in enumerate(state_labels)}
print("\nMapping state_numeric -> state:")
for k, v in numeric_to_state.items():
    print(f"{k}: {v}")

# Hiển thị thống kê mô tả
df.describe().show()


In [ ]:
# Xử lý outlier cho cột price
q95 = df.approxQuantile("price", [0.95], 0.01)[0]
q25 = df.approxQuantile("price", [0.25], 0.01)[0]
iqrMax = q95 + q25
print(f"\nThreshold cho price: {iqrMax}")
percent_outliers = df.filter(col("price") > 3150000.0).count() / df.count() * 100
print(f"Tỷ lệ outliers trong price: {percent_outliers:.2f}%")
df = df.filter(col("price") <= 3150000.0)

# Xử lý outlier cho acre_lot
percent_outliers = df.filter(col("acre_lot") > 200).count() / df.count() * 100
print(f"\nTỷ lệ outliers trong acre_lot: {percent_outliers:.2f}%")
df = df.filter(col("acre_lot") <= 200)

# Xử lý outlier cho house_size
percent_outliers = df.filter(col("house_size") >= 20000).count() / df.count() * 100
print(f"\nTỷ lệ outliers trong house_size: {percent_outliers:.2f}%")
df = df.filter(col("house_size") < 20000)

# Kiểm tra lại giá trị null sau khi xử lý outliers
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
print("\nSố lượng giá trị null sau khi xử lý outliers:")
null_counts.show()

# Hiển thị thống kê mô tả
df.describe().show()

In [ ]:
import xgboost as xgb
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id, when

print("\nĐang điền giá trị thiếu cho acre_lot bằng XGBoost...")

# Kiểm tra nếu có giá trị thiếu
if df.filter(col("acre_lot").isNull()).count() > 0:

    # Chuyển dữ liệu có acre_lot sang Pandas
    filled_pdf = filled_df.toPandas()
    missing_pdf = missing_df.toPandas()

    # Kiểm tra nếu có dữ liệu để huấn luyện
    if not filled_pdf.empty and not missing_pdf.empty:
        # Chuẩn bị dữ liệu huấn luyện
        X_train_fill = filled_pdf.drop(columns=["acre_lot"])
        y_train_fill = filled_pdf["acre_lot"]

        X_test_fill = missing_pdf.drop(columns=["acre_lot"])

        # Huấn luyện mô hình XGBoost
        model_fill = xgb.XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            subsample=0.8,
            colsample_bytree=0.8
        )
        model_fill.fit(X_train_fill, y_train_fill)

        # Dự đoán giá trị thiếu
        preds = model_fill.predict(X_test_fill)

        # Chuyển kết quả về DataFrame PySpark
        preds_df = pd.DataFrame({"acre_lot": preds})
        preds_spark = spark.createDataFrame(preds_df)

        # Thêm ID để kết hợp lại dữ liệu
        df = df.withColumn("id", monotonically_increasing_id())
        missing_df = missing_df.withColumn("id", monotonically_increasing_id())
        preds_spark = preds_spark.withColumn("id", monotonically_increasing_id())

        # Kết hợp giá trị dự đoán vào tập dữ liệu gốc
        df = df.join(preds_spark, "id", "left_outer").drop("id")
        df = df.withColumn("acre_lot", when(col("acre_lot").isNull(), col("prediction")).otherwise(col("acre_lot"))).drop("prediction")

        print(f"Đã điền {len(preds)} giá trị thiếu cho acre_lot bằng XGBoost!")

    else:
        print("Không có đủ dữ liệu để huấn luyện XGBoost. Bỏ qua việc điền giá trị thiếu.")

else:
    print("Không có giá trị thiếu trong acre_lot. Không cần điền.")



In [ ]:
import xgboost as xgb
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id, when

print("\nĐang điền giá trị thiếu cho house_size bằng XGBoost...")

# Kiểm tra nếu có giá trị thiếu
if df.filter(col("house_size").isNull()).count() > 0:

    # Chuyển dữ liệu PySpark sang Pandas
    filled_pdf = df.filter(col("house_size").isNotNull()).toPandas()
    missing_pdf = df.filter(col("house_size").isNull()).toPandas()

    # Kiểm tra nếu có đủ dữ liệu để huấn luyện
    if not filled_pdf.empty and not missing_pdf.empty:
        # Chuẩn bị dữ liệu huấn luyện
        X_train_fill = filled_pdf.drop(columns=["house_size"])
        y_train_fill = filled_pdf["house_size"]

        X_test_fill = missing_pdf.drop(columns=["house_size"])

        # Xử lý giá trị NaN bằng giá trị trung bình
        X_train_fill = X_train_fill.fillna(X_train_fill.mean())
        X_test_fill = X_test_fill.fillna(X_train_fill.mean())

        # Huấn luyện mô hình XGBoost
        model_fill = xgb.XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            subsample=0.8,
            colsample_bytree=0.8,
            tree_method='hist'
        )
        model_fill.fit(X_train_fill, y_train_fill)

        # Dự đoán giá trị thiếu
        preds = model_fill.predict(X_test_fill)

        # Chuyển kết quả về DataFrame PySpark
        preds_df = pd.DataFrame({"house_size": preds})
        preds_spark = spark.createDataFrame(preds_df)

        # Thêm ID để kết hợp lại dữ liệu
        df = df.withColumn("id", monotonically_increasing_id())
        missing_df = df.filter(col("house_size").isNull()).withColumn("id", monotonically_increasing_id())
        preds_spark = preds_spark.withColumn("id", monotonically_increasing_id())

        # Kết hợp giá trị dự đoán vào tập dữ liệu gốc
        df = df.join(preds_spark, "id", "left_outer").drop("id")
        df = df.withColumn("house_size", when(col("house_size").isNull(), col("prediction")).otherwise(col("house_size"))).drop("prediction")

        print(f"Đã điền {len(preds)} giá trị thiếu cho house_size bằng XGBoost!")

    else:
        print("Không có đủ dữ liệu để huấn luyện XGBoost. Bỏ qua việc điền giá trị thiếu.")

else:
    print("Không có giá trị thiếu trong house_size. Không cần điền.")


In [ ]:
from pyspark.sql.functions import col, when, expr

print("Đang tạo các đặc trưng mới...")

# Tránh chia cho 0 bằng cách sử dụng F.when
df = df.withColumn("bed_bath_ratio", col("bed") / when(col("bath") == 0, 1).otherwise(col("bath")))
df = df.withColumn("total_rooms", col("bed") + col("bath"))
df = df.withColumn("room_density", col("total_rooms") / when(col("house_size") == 0, 1).otherwise(col("house_size")))
df = df.withColumn("house_size_per_bed", col("house_size") / when(col("bed") == 0, 1).otherwise(col("bed")))
df = df.withColumn("house_size_per_bath", col("house_size") / when(col("bath") == 0, 1).otherwise(col("bath")))
df = df.withColumn("lot_to_house_ratio", col("acre_lot") / when(col("house_size") == 0, 1).otherwise(col("house_size")))
df = df.withColumn("size_by_state", col("house_size") * col("state_numeric"))
df = df.withColumn("rooms_by_state", col("total_rooms") * col("state_numeric"))

# Điền giá trị thiếu cho các cột số (tránh lỗi)
numeric_columns = [c for c, t in df.dtypes if t in ('int', 'double')]
df = df.fillna(0, subset=numeric_columns)

print("Hoàn tất tạo đặc trưng mới!")


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

print("\nChuẩn bị dữ liệu cho mô hình...")

# Xác định cột đặc trưng (bỏ cột price)
feature_cols = [c for c in df.columns if c != "price"]

# Dùng VectorAssembler để gộp tất cả đặc trưng vào một cột duy nhất
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df).select("features", "price")

# Chia dữ liệu thành tập huấn luyện và kiểm tra (80/20)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

print(f"Kích thước tập huấn luyện: {train_df.count()} mẫu, tập kiểm tra: {test_df.count()} mẫu")


In [ ]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator

print("\nBắt đầu tìm kiếm siêu tham số tối ưu...")

# Sử dụng GBTRegressor với các tham số được tối ưu
gbt = GBTRegressor(
    featuresCol="features", 
    labelCol="price",
    maxBins=32,        # Tăng maxBins để cải thiện hiệu suất
    maxIter=100,       # Số lần lặp
    stepSize=0.1,      # Learning rate
    maxDepth=5,        # Độ sâu cây
    subsamplingRate=0.8  # Subsampling rate
)

# Giảm tham số trong lưới tìm kiếm để tránh timeout
param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [3]) \
    .addGrid(gbt.stepSize, [0.1]) \
    .build()

# Đánh giá mô hình
evaluator = RegressionEvaluator(
    labelCol="price", 
    predictionCol="prediction", 
    metricName="r2"
)

# Sử dụng TrainValidationSplit với tỷ lệ train cao hơn
tvs = TrainValidationSplit(
    estimator=gbt,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    trainRatio=0.9,  # 90% huấn luyện, 10% kiểm định
    seed=42
)

# Huấn luyện mô hình
print("Đang huấn luyện mô hình, vui lòng đợi...")
model = tvs.fit(train_df)

# Lấy mô hình tốt nhất
best_model = model.bestModel
print("\nĐã tìm thấy mô hình tốt nhất!")
print(f"Best maxDepth: {best_model.getMaxDepth()}")
print(f"Best maxIter: {best_model.getMaxIter()}")
print(f"Best stepSize: {best_model.getStepSize()}")



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

print("\nĐánh giá mô hình trên tập kiểm tra...")

# Dự đoán
predictions = best_model.transform(test_df)

# Khởi tạo bộ đánh giá
evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
evaluator_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")

# Tính toán các chỉ số đánh giá
r2 = evaluator_r2.evaluate(predictions)
rmse = evaluator_rmse.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)

print("\n🔍 Đánh giá mô hình trên dữ liệu thực tế:")
print(f"✅ R² Score: {r2:.4f}")
print(f"✅ RMSE: ${rmse:.2f}")
print(f"✅ MAE: ${mae:.2f}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print("Đang vẽ biểu đồ...")

# Chuyển dữ liệu PySpark về Pandas
predictions_pd = predictions.select("price", "prediction").toPandas()

# 1. Biểu đồ dự đoán vs thực tế
plt.figure(figsize=(12, 8), clear=True)
plt.scatter(predictions_pd["price"], predictions_pd["prediction"], alpha=0.5)
plt.plot(
    [predictions_pd["price"].min(), predictions_pd["price"].max()],
    [predictions_pd["price"].min(), predictions_pd["price"].max()],
    'r--'
)
plt.title('Giá trị dự đoán vs Thực tế', fontsize=15)
plt.xlabel('Giá trị thực tế ($)', fontsize=12)
plt.ylabel('Giá trị dự đoán ($)', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

# 2. Biểu đồ tầm quan trọng của đặc trưng
plt.figure(figsize=(12, 8), clear=True)
feature_importance = best_model.featureImportances.toArray()  # Chuyển sang mảng numpy
feature_names = X.columns
indices = np.argsort(feature_importance)[::-1]
top_n = min(10, len(feature_names))

plt.barh(range(top_n), feature_importance[indices][:top_n], align='center')
plt.yticks(range(top_n), [feature_names[i] for i in indices][:top_n])
plt.xlabel('Tầm quan trọng', fontsize=12)
plt.title('Top 10 đặc trưng quan trọng nhất', fontsize=15)
plt.gca().invert_yaxis()  # Đảo ngược trục để đặc trưng quan trọng nhất ở trên
plt.tight_layout()
plt.show()
